In [1]:
import json
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
# print(os.getcwd())
import dist_utilities as dist_ult
# import noise_est_utilities as ne_ult
importlib.reload(dist_ult)
os.chdir(current_dir)
import pickle
from qiskit.result import Result
from qiskit.visualization import plot_histogram, plot_distribution
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import numpy as np
from copy import deepcopy
from qiskit.result import QuasiDistribution

In [2]:
with open("ideal_counts_hardware.json", "r") as f:
    ideal_counts=json.load(f)
with open("noisy_counts_hardware.json", "r") as f:
    noisy_counts=json.load(f)
with open("noisy_noise_est_dist_hardware.json", "r") as f:
    noisy_noise_est_dist=json.load(f)
with open("ideal_noise_est_counts_hardware.json", "r") as f:
    ideal_noise_est_counts=json.load(f)
print("ideal: ", ideal_noise_est_counts)
print("noise est dist: ", noisy_noise_est_dist)
print("noisy counts: ", noisy_counts)
print("ideal counts: ", ideal_counts)
col_label=list(dict(ideal_noise_est_counts).keys())[0]
# corrected_counts=ne_ult.get_corrected_counts_from_counts(noisy_counts, noisy_noise_est_dist, list(ideal_noise_est_counts.keys())[0])

# print(corrected_counts)

ideal:  {'101100': 200000}
noise est dist:  {'001100': 0.047045, '101100': 0.64813, '101110': 0.082235, '100100': 0.05038, '101000': 0.03217, '111100': 0.032915, '000100': 0.004925, '101101': 0.029565, '111110': 0.004525, '001110': 0.00972, '000101': 0.00023, '111001': 0.00055, '101111': 0.003565, '011100': 0.002305, '001101': 0.00206, '001111': 0.00035, '100101': 0.0026, '100110': 0.008435, '000110': 0.00223, '101010': 0.004025, '100000': 0.00468, '111101': 0.001995, '100010': 0.00076, '110100': 0.005325, '111000': 0.003855, '110110': 0.00084, '111011': 6e-05, '101001': 0.003295, '110000': 0.00197, '101011': 0.000415, '100111': 0.00044, '010100': 0.00045, '001001': 0.000175, '111010': 0.000525, '110111': 5.5e-05, '001010': 0.000535, '001000': 0.00258, '111111': 0.00021, '100001': 0.00051, '110101': 0.00041, '110001': 0.00021, '011110': 0.000515, '110010': 0.000325, '000000': 0.000375, '000111': 0.0001, '011000': 0.00027, '010110': 0.0002, '010010': 4e-05, '011101': 0.000145, '010000':

In [3]:
# plot_histogram([ideal_counts, noisy_counts, corrected_counts], number_to_keep=5, legend=["ideal counts", "noisy counts", "corrected counts"], figsize=(20,20), filename="plot_sim.jpg")


In [4]:
num_qubits=len(list(noisy_counts.items())[0][0])

# if 2**10<2**num_qubits:
num_keep=2**15#2**10#num_qubits#2**10
if num_keep>2**num_qubits:
    num_keep=2**num_qubits
noisy_counts_orig=noisy_counts
print("num keep ", num_keep)
noisy_noise_est_dist=dist_ult.convert_to_first_col(noisy_noise_est_dist, col_label)
noisy_noise_est_dist=dist_ult.filter_counts(noisy_noise_est_dist, num_keep)
noisy_counts=dist_ult.filter_counts(noisy_counts, num_keep)
all_keys, noisy_noise_est_dist, noisy_counts=dist_ult.fill_counts_range(noisy_noise_est_dist, noisy_counts, num_keep, num_qubits)

noisy_noise_est_vec=sorted(list(noisy_noise_est_dist.items()), key=lambda x: int(x[0], 2))
noisy_noise_est_vec=[elem[1] for elem in noisy_noise_est_vec]
counts_vec=sorted(list(noisy_counts.items()), key=lambda x: int(x[0], 2))
counts_vec=[elem[1] for elem in counts_vec]
# print(len(noisy_noise_est_vec))
# print(len(counts_vec))
quasi_corrected_counts=dist_ult.get_quasi_corrected(noisy_noise_est_vec, counts_vec)
quasi_corrected_counts=dist_ult.label_quasis(quasi_corrected_counts, all_keys)
quasi_method=QuasiDistribution(quasi_corrected_counts)
quasi_corrected_counts=quasi_method.nearest_probability_distribution().binary_probabilities(num_qubits)
# print(quasi_corrected_counts)
# quasi_corrected_counts={bin(idx)[2::].zfill(num_qubits):val for idx, val in enumerate(quasi_corrected_counts)}
elem_sorted=sorted(quasi_corrected_counts.items(), key=lambda x: x[1])[::-1]
print(elem_sorted[0:5])



num keep  64
max exist  63
min exist  0
size before  64
64
64
[('111111', 179402.291008155), ('000000', 5506.46548007298), ('100001', 3740.14272886970), ('100111', 3017.83513395761), ('100000', 2952.24119658335)]


In [5]:
fig=plot_histogram([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=5, 
               legend=["ideal", "raw", "corrected"], title="GQuantum Phase Estimation 6 Qubits Quantum Hardware")
fig.savefig("plot_qpe6_counts_hardware_ibm_marrakesh.jpg", bbox_inches="tight")


In [6]:
fig=plot_distribution([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=5, 
                  legend=["ideal", "raw", "corrected"], title="Quantum Phase Estimation 6 Qubits Quantum Hardware") #figsize=(15,15))
ax=fig.axes[0]
ax.set_ylabel("Probability")
fig.savefig("plot_qpe6_dist_hardware_ibm_marrakesh.jpg", bbox_inches="tight")

In [7]:
from qiskit.quantum_info import hellinger_fidelity
print(hellinger_fidelity(ideal_counts, quasi_corrected_counts))
print(hellinger_fidelity(ideal_counts, noisy_counts_orig))
print(ideal_counts)
print(quasi_corrected_counts)
print(noisy_counts_orig)

0.8970114550407747
0.57917
{'111111': 200000}
{'000001': 16.4498437200757, '000010': 70.6449690603665, '101101': 79.9169682249023, '110011': 119.675096086908, '111100': 130.898417241235, '111001': 169.176261072738, '101011': 326.589985313976, '100101': 345.421942197995, '100010': 530.666471225217, '011110': 692.211914535389, '111110': 699.400488409211, '100011': 2199.97209527338, '100000': 2952.24119658335, '100111': 3017.83513395761, '100001': 3740.14272886970, '000000': 5506.46548007298, '111111': 179402.291008155}
{'111111': 115834, '100000': 2885, '011111': 5016, '101111': 6039, '111101': 13842, '100001': 3344, '110111': 3678, '110011': 1044, '011110': 1207, '000000': 4263, '110000': 390, '101101': 1327, '110101': 957, '111011': 3413, '100111': 3245, '111110': 6309, '011010': 325, '110110': 640, '101001': 586, '111100': 1277, '010100': 255, '000011': 491, '100100': 458, '101011': 1397, '011101': 1136, '111001': 958, '000110': 286, '010010': 278, '000001': 885, '010110': 209, '11100